In [ ]:
# Clone the dataset from GitHub
!git clone https://github.com/PUTvision/UAVVaste.git

# Navigate into the dataset directory
%cd UAVVaste

Cloning into 'UAVVaste'...
remote: Enumerating objects: 81, done.
remote: Counting objects: 100% (81/81), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 81 (delta 36), reused 41 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (81/81), 427.63 KiB | 639.00 KiB/s, done.
Resolving deltas: 100% (36/36), done.
/content/UAVVaste


In [ ]:
# Install any necessary libraries
!pip install Pillow

In [ ]:
import json
import os
import requests

# Paths
annotations_path = "/content/UAVVaste/annotations/annotations.json"
images_dir = "/content/UAVVaste/images"

# Create images directory if not exists
os.makedirs(images_dir, exist_ok=True)

# Load annotations
with open(annotations_path, "r") as f:
    data = json.load(f)

# Download images
for img_info in data["images"]:
    img_name = img_info["file_name"]
    img_url = img_info["flickr_url"]

    if img_url:  # Check if a URL is provided
        img_path = os.path.join(images_dir, img_name)
        response = requests.get(img_url, stream=True)

        if response.status_code == 200:
            with open(img_path, "wb") as img_file:
                img_file.write(response.content)
            print(f"Downloaded: {img_name}")
        else:
            print(f"Failed to download: {img_name}")

print("All images downloaded successfully!")


Downloaded: BATCH_d07_img_6400.jpg
Downloaded: BATCH_d07_img_580.jpg
Downloaded: BATCH_d07_img_2380.jpg
Downloaded: BATCH_d07_img_880.jpg
Downloaded: BATCH_d07_img_4550.jpg
Downloaded: BATCH_d07_img_410.jpg
Downloaded: BATCH_d07_img_6030.jpg
Downloaded: BATCH_d07_img_3470.jpg
Downloaded: BATCH_d07_img_6970.jpg
Downloaded: BATCH_d07_img_5110.jpg
Downloaded: BATCH_d07_img_1250.jpg
Downloaded: BATCH_d07_img_450.jpg
Downloaded: BATCH_d07_img_2760.jpg
Downloaded: BATCH_d07_img_2340.jpg
Downloaded: BATCH_d07_img_610.jpg
Downloaded: BATCH_d07_img_2080.jpg
Downloaded: BATCH_d07_img_7260.jpg
Downloaded: BATCH_d07_img_2930.jpg
Downloaded: BATCH_d07_img_4370.jpg
Downloaded: BATCH_d07_img_1760.jpg
Downloaded: BATCH_d07_img_4730.jpg
Downloaded: BATCH_d07_img_1700.jpg
Downloaded: BATCH_d07_img_4670.jpg
Downloaded: BATCH_d07_img_1390.jpg
Downloaded: BATCH_d07_img_360.jpg
Downloaded: BATCH_d07_img_600.jpg
Downloaded: BATCH_d07_img_1120.jpg
Downloaded: BATCH_d07_img_510.jpg
Downloaded: BATCH_d07_img_65

In [ ]:
import cv2
import numpy as np
import json
from pathlib import Path
from pycocotools.coco import COCO
from tqdm import tqdm

def segmentation_to_bbox(segmentation, img_w, img_h):
    # Segmentation is a list of x, y coordinates for the polygon
    x_coords = segmentation[::2]  # Extract all x coordinates (even indices)
    y_coords = segmentation[1::2]  # Extract all y coordinates (odd indices)

    # Find the minimum and maximum values for x and y to create a bounding box
    xmin = min(x_coords)
    xmax = max(x_coords)
    ymin = min(y_coords)
    ymax = max(y_coords)

    # Calculate the center of the bounding box (normalized)
    center_x = (xmin + xmax) / 2 / img_w
    center_y = (ymin + ymax) / 2 / img_h

    # Calculate the width and height of the bounding box (normalized)
    width = (xmax - xmin) / img_w
    height = (ymax - ymin) / img_h

    return center_x, center_y, width, height

def convert_json_to_yolo_mask(annotation_path: str, images_dir_path: str, dest_path: str, train_val_test: dict):
    dataset = COCO(annotation_path)

    for img_id in tqdm(dataset.imgs):
        img_info = dataset.imgs[img_id]
        img_name = img_info['file_name']
        img_path = Path(images_dir_path) / img_name
        img = cv2.imread(str(img_path))

        # Check if image is loaded correctly
        if img is None:
            print(f"Warning: Couldn't read image {img_name}")
            continue

        img_h, img_w, _ = img.shape

        ann_ids = dataset.getAnnIds(imgIds=img_id)
        anns = dataset.loadAnns(ann_ids)

        # Determine the destination directory based on train/val/test split
        if img_name in train_val_test['train']:
            dest_dir = Path(dest_path) / 'train'
        elif img_name in train_val_test['val']:
            dest_dir = Path(dest_path) / 'val'
        elif img_name in train_val_test['test']:
            dest_dir = Path(dest_path) / 'test'

        dest_dir.mkdir(parents=True, exist_ok=True)

        # Save the image
        cv2.imwrite(str(dest_dir / img_name), img)

        # Create the corresponding YOLO text file
        with open(str(dest_dir / img_name).replace('.jpg', '.txt').replace('.JPG', '.txt').replace('.PNG', '.txt'), 'w') as f:
            for ann in anns:
                row = '0'  # Assuming class_id is 0 for all objects, adjust if needed

                # Use the segmentation_to_bbox function to convert segmentation to bounding box
                bbox = segmentation_to_bbox(ann['segmentation'][0], img_w, img_h)

                # Add the normalized bounding box coordinates to the row
                row += f' {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}'

                # Write the row for each object in the image
                row += '\n'
                f.write(row)


if __name__ == '__main__':
    # Update paths here with your specific directories
    annotation_path = '/content/UAVVaste/annotations/annotations.json'  # Path to your annotations file
    images_dir_path = '/content/UAVVaste/images'  # Path to your images directory
    dest_path = '/content/UAVVaste/yolo_annotations/'  # Path where YOLO format files will be saved

    # Load train, val, and test splits
    with open('/content/UAVVaste/annotations/train_val_test_distribution_file.json', 'r') as f:
        train_val_test = json.load(f)

    convert_json_to_yolo_mask(annotation_path, images_dir_path, dest_path, train_val_test)


loading annotations into memory...
Done (t=0.02s)
creating index...
index created!


100%|██████████| 772/772 [01:43<00:00,  7.49it/s]


In [ ]:
from pathlib import Path

image_dir = Path('/content/UAVVaste/images/')
missing_images = []

with open('/content/UAVVaste/annotations/train_val_test_distribution_file.json', 'r') as f:
    train_val_test = json.load(f)

all_images = train_val_test['train'] + train_val_test['val'] + train_val_test['test']

for img_name in all_images:
    img_path = image_dir / img_name
    if not img_path.exists():
        missing_images.append(img_name)

print(f"Missing images: {missing_images}")


Missing images: []


In [ ]:
import os
import shutil
from pathlib import Path

def move_files(src_dirs, dest_images_dir, dest_labels_dir):
    # Create destination directories if they don't exist
    Path(dest_images_dir).mkdir(parents=True, exist_ok=True)
    Path(dest_labels_dir).mkdir(parents=True, exist_ok=True)

    # Loop through the source directories (train, test, val)
    for src_dir in src_dirs:
        # Ensure the source directory exists
        if not os.path.exists(src_dir):
            print(f"Warning: {src_dir} doesn't exist. Skipping.")
            continue

        # List all files in the source directory
        for file in os.listdir(src_dir):
            file_path = os.path.join(src_dir, file)

            # If it's an image, move to the 'images' folder
            if file.lower().endswith(('.jpg', '.jpeg', '.png', '.JPG', '.JPEG', '.PNG')):
                dest_file = os.path.join(dest_images_dir, file)
                shutil.move(file_path, dest_file)
                print(f"Moved image: {file}")

            # If it's a text file, move to the 'labels' folder
            elif file.lower().endswith('.txt'):
                dest_file = os.path.join(dest_labels_dir, file)
                shutil.move(file_path, dest_file)
                print(f"Moved label: {file}")

if __name__ == '__main__':
    # Paths to your source directories (train, val, test)
    src_dirs = [
        '/content/UAVVaste/yolo_annotations/train',
        '/content/UAVVaste/yolo_annotations/val',
        '/content/UAVVaste/yolo_annotations/test'
    ]

    # Destination directories for images and labels
    dest_images_dir = '/content/data/images'
    dest_labels_dir = '/content/data/labels'

    # Move files
    move_files(src_dirs, dest_images_dir, dest_labels_dir)


Moved label: BATCH_s04_img_3460.txt
Moved image: GOPR0044.JPG
Moved image: batch_05_img_1540.jpg
Moved image: BATCH_s05_img_210.jpg
Moved label: BATCH_d07_img_2340.txt
Moved image: batch_s02_img_150.jpg
Moved image: BATCH_d06_img_2260.jpg
Moved image: BATCH_s04_img_3600.jpg
Moved image: BATCH_d08_img_5300.jpg
Moved label: batch_s02_img_16.txt
Moved image: BATCH_s05_img_1940.jpg
Moved label: GOPR0041.txt
Moved label: batch_04_img_1780.txt
Moved label: BATCH_d06_img_3280.txt
Moved image: BATCH_s03_img_2080.jpg
Moved image: batch_s02_img_120.jpg
Moved label: BATCH_d06_img_2260.txt
Moved image: batch_s02_img_89.jpg
Moved image: batch_05_img_1820.jpg
Moved image: BATCH_d06_img_2840.jpg
Moved image: batch_02_img_320.jpg
Moved image: batch_s01_img_2740.jpg
Moved image: batch_03_img_3740.jpg
Moved image: BATCH_d07_img_5710.jpg
Moved image: batch_s02_img_12.jpg
Moved image: BATCH_d07_img_2670.jpg
Moved image: BATCH_d07_img_720.jpg
Moved image: BATCH_d07_img_810.jpg
Moved image: BATCH_d07_img_16

In [ ]:
!wget -O /content/train_val_split.py https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py

# TO DO: Improve robustness of train_val_split.py script so it can handle nested data folders, etc
!python /content/train_val_split.py --datapath="/content/data" --train_pct=0.9


--2025-03-31 19:21:43--  https://raw.githubusercontent.com/EdjeElectronics/Train-and-Deploy-YOLO-Models/refs/heads/main/utils/train_val_split.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3203 (3.1K) [text/plain]
Saving to: ‘/content/train_val_split.py’

/content/train_val_ 100%[===================>]   3.13K  --.-KB/s    in 0s      

2025-03-31 19:21:43 (41.4 MB/s) - ‘/content/train_val_split.py’ saved [3203/3203]

Created folder at /content/UAVVaste/data/train/images.
Created folder at /content/UAVVaste/data/train/labels.
Created folder at /content/UAVVaste/data/validation/images.
Created folder at /content/UAVVaste/data/validation/labels.
Number of image files: 772
Number of annotation files: 772
Images moving to train: 694
Images moving to validation

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 976.9/976.9 kB 51.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 87.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 70.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 48.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 69.4 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstall

In [ ]:
# Python function to automatically create data.yaml config file
# 1. Reads "classes.txt" file to get list of class names
# 2. Creates data dictionary with correct paths to folders, number of classes, and names of classes
# 3. Writes data in YAML format to data.yaml

import yaml
import os

def create_data_yaml(path_to_classes_txt, path_to_data_yaml):

  # Read class.txt to get class names
  if not os.path.exists(path_to_classes_txt):
    print(f'classes.txt file not found! Please create a classes.txt labelmap and move it to {path_to_classes_txt}')
    return
  with open(path_to_classes_txt, 'r') as f:
    classes = []
    for line in f.readlines():
      if len(line.strip()) == 0: continue
      classes.append(line.strip())
  number_of_classes = len(classes)

  # Create data dictionary
  data = {
      'path': '/content/UAVVaste/data',
      'train': 'train/images',
      'val': 'validation/images',
      'nc': number_of_classes,
      'names': classes
  }

  # Write data to YAML file
  with open(path_to_data_yaml, 'w') as f:
    yaml.dump(data, f, sort_keys=False)
  print(f'Created config file at {path_to_data_yaml}')

  return

# Define path to classes.txt and run function
path_to_classes_txt = '/content/UAVVaste/data/classes.txt'
path_to_data_yaml = '/content/UAVVaste/data.yaml'

create_data_yaml(path_to_classes_txt, path_to_data_yaml)

print('\nFile contents:\n')
!cat /content/UAVVaste/data.yaml

Created config file at /content/UAVVaste/data.yaml

File contents:

path: /content/UAVVaste/data
train: train/images
val: validation/images
nc: 1
names:
- '0'


In [ ]:
!yolo detect train data=/content/UAVVaste/data.yaml model=yolo11s.pt epochs=100 imgsz=640 patience=70 resume=True

Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11s.pt, data=/content/UAVVaste/data.yaml, epochs=70, time=None, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train2, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxe

In [ ]:
!yolo detect predict model=runs/detect/train2/weights/best.pt source=data/validation/images save=True

Ultralytics 8.3.99 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
YOLO11s summary (fused): 100 layers, 9,413,187 parameters, 0 gradients, 21.3 GFLOPs

image 1/78 /content/UAVVaste/data/validation/images/BATCH_d06_img_0.jpg: 384x640 3 0s, 69.1ms
image 2/78 /content/UAVVaste/data/validation/images/BATCH_d06_img_130.jpg: 384x640 4 0s, 12.1ms
image 3/78 /content/UAVVaste/data/validation/images/BATCH_d06_img_1790.jpg: 384x640 2 0s, 17.0ms
image 4/78 /content/UAVVaste/data/validation/images/BATCH_d06_img_2540.jpg: 384x640 4 0s, 18.4ms
image 5/78 /content/UAVVaste/data/validation/images/BATCH_d06_img_3100.jpg: 384x640 6 0s, 17.0ms
image 6/78 /content/UAVVaste/data/validation/images/BATCH_d07_img_1010.jpg: 384x640 1 0, 14.8ms
image 7/78 /content/UAVVaste/data/validation/images/BATCH_d07_img_1330.jpg: 384x640 4 0s, 10.9ms
image 8/78 /content/UAVVaste/data/validation/images/BATCH_d07_img_1650.jpg: 384x640 10 0s, 10.9ms
image 9/78 /content/UAVVaste/data/validation/images/BATCH_d07

In [ ]:
import glob
from IPython.display import Image, display
for image_path in glob.glob(f'/content/runs/detect/predict/*.jpg')[:10]:
  display(Image(filename=image_path, height=400))
  print('\n')